In [34]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage
#pio.templates.default = "plotly_white"

In [4]:
# import training dataset
d1 = pd.read_csv("../../ML_datasets/QSAR_Molecular_Structure_Predictions/train_rows.csv")

In [5]:
print(d1.head())

   SpMax_L  J_Dz(e)  nHM  F01[N-N]  F04[C-N]  NssssC  nCb-    C%  nCp  nO  \
0    3.919   2.6909    0         0         0       0     0  31.4    2   0   
1    4.170   2.1144    0         0         0       0     0  30.8    1   1   
2    3.932   3.2512    0         0         0       0     0  26.7    2   4   
3    3.000   2.7098    0         0         0       0     0  20.0    0   2   
4    4.236   3.3944    0         0         0       0     0  29.4    2   4   

   ...  C-026  F02[C-N]  nHDon  SpMax_B(m)  Psi_i_A  nN  SM6_B(m)  nArCOOR  \
0  ...      0         0      0       2.949    1.591   0     7.253        0   
1  ...      0         0      0       3.315    1.967   0     7.257        0   
2  ...      0         0      1       3.076    2.417   0     7.601        0   
3  ...      0         0      1       3.046    5.000   0     6.690        0   
4  ...      0         0      0       3.351    2.405   0     8.003        0   

   nX  Class  
0   0      1  
1   0      1  
2   0      1  
3   0   

In [6]:
features = d1.columns
print(features)

Index(['SpMax_L', 'J_Dz(e)', 'nHM', 'F01[N-N]', 'F04[C-N]', 'NssssC', 'nCb-',
       'C%', 'nCp', 'nO', 'F03[C-N]', 'SdssC', 'HyWi_B(m)', 'LOC', 'SM6_L',
       'F03[C-O]', 'Me', 'Mi', 'nN-N', 'nArNO2', 'nCRX3', 'SpPosA_B(p)',
       'nCIR', 'B01[C-Br]', 'B03[C-Cl]', 'N-073', 'SpMax_A', 'Psi_i_1d',
       'B04[C-Br]', 'SdO', 'TI2_L', 'nCrt', 'C-026', 'F02[C-N]', 'nHDon',
       'SpMax_B(m)', 'Psi_i_A', 'nN', 'SM6_B(m)', 'nArCOOR', 'nX', 'Class'],
      dtype='object')


In [30]:
# 1) Find the correlation between features and plot them
# Compute the correlation matrix
correlation_matrix = d1.corr()

# Extract all pairwise correlations
correlations = correlation_matrix.unstack()

# Plot the heatmap using Plotly
fig = px.imshow(
    correlation_matrix,
    text_auto=".2f",
    #color_continuous_scale="coolwarm",
    title="Correlation Matrix Heatmap",
    labels={"color": "Correlation"},
)

# Customize layout
fig.update_layout(
    xaxis=dict(title="Features", tickangle=45),
    yaxis=dict(title="Features"),
    title_x=0.5  # Center the title
)

fig.show()


In [31]:
# 2) Top 50 correlations
# Remove self-correlations (e.g., corr(a, a)) and duplicate pairs (e.g., corr(a, b) = corr(b, a))
# Create a condition to ensure no duplicates (e.g., corr(a, b) will be considered once)
correlations = correlations[correlations.index.get_level_values(0) != correlations.index.get_level_values(1)]
correlations = correlations[correlations.index.get_level_values(0) < correlations.index.get_level_values(1)]

# Sort correlations by absolute value
correlations = correlations.abs().sort_values(ascending=False)

# Get the top 50 correlations
top_50 = correlations.head(50).reset_index()
top_50.columns = ['Feature1', 'Feature2', 'Correlation']

# Plot the top 50 correlations obeseved between features in a bar plot
fig = px.bar(
    top_50,
    x='Correlation',
    y=top_50.index,
    color='Correlation',
    orientation='h',
    title="Top 50 Feature Correlations",
    labels={"y": "Feature Pair Index", "Correlation": "Correlation Coefficient"}
)
fig.update_layout(yaxis=dict(tickmode='array', tickvals=top_50.index, ticktext=[f"{row['Feature1']} vs {row['Feature2']}" for _, row in top_50.iterrows()]))

fig.show()


In [32]:
# 3) Scatter plots for each pair
# Create a 10x5 grid layout
fig = make_subplots(
    rows=10,
    cols=5,
    subplot_titles=[
        f"{row['Feature1']} vs {row['Feature2']}<br>Correlation: {row['Correlation']:.2f}"
        for _, row in top_50.iterrows()
    ]
)

# Add scatter plots for each pair
for idx, row in top_50.iterrows():
    feature1 = row['Feature1']
    feature2 = row['Feature2']
    correlation = row['Correlation']

    # Determine row and column for the subplot
    subplot_row = idx // 5 + 1
    subplot_col = idx % 5 + 1

    # Add the scatter plot
    fig.add_trace(
        go.Scatter(
            x=d1[feature1],
            y=d1[feature2],
            mode='markers',
            name=f"{feature1} vs {feature2}",
            marker=dict(size=4, opacity=0.7)
        ),
        row=subplot_row,
        col=subplot_col
    )

    # Update axes titles dynamically
    fig.update_xaxes(title_text=f"{feature1}", row=subplot_row, col=subplot_col)
    fig.update_yaxes(title_text=f"{feature2}", row=subplot_row, col=subplot_col)

# Update layout
fig.update_layout(
    height=2500,  # Adjust height for readability
    width=2000,   # Adjust width for readability
    title_text="Scatter Plots for Top 50 Correlations",
    title_x=0.5,
    showlegend=False,
)

fig.show()

In [23]:
# 4) PCA to determine top features
# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(d1)

# Apply PCA
pca = PCA(n_components=18)  # Keep all components for analysis
pca_result = pca.fit(scaled_data)

# Explained variance ratio for each component
explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

# Create PCA loadings table
loadings = pd.DataFrame(
    pca.components_.T,  # Transpose to align features with components
    index=d1.columns,   # Feature names
    columns=[f"PC{i+1}" for i in range(len(explained_variance))]
)

# Create a DataFrame for visualization
pca_df = pd.DataFrame({
    "Principal Component": [f"PC{i+1}" for i in range(len(explained_variance))],
    "Explained Variance": explained_variance,
    "Cumulative Variance": cumulative_variance
})

# Create a table to show the top contributing features for each PC
loadings_table = pd.DataFrame({
    "Principal Component": [f"PC{i+1}" for i in range(len(explained_variance))],
    "Top Contributing Feature": loadings.idxmax(axis=0).values  # Feature with max loading per PC
})

# Print the loadings table
print("Top Contributing Features per Principal Component:")
print(loadings_table)

# Plot explained variance ratio
fig = px.bar(
    pca_df,
    x="Principal Component",
    y="Explained Variance",
    text="Explained Variance",
    title="Explained Variance by Principal Components",
    labels={"Explained Variance": "Variance Ratio"}
)
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(yaxis=dict(title="Variance Ratio"), xaxis=dict(title="Principal Component"))
fig.show()

# Transform data into top components (e.g., top 2 components)
top_components = 2
transformed_data = pca.transform(scaled_data)[:, :top_components]

# Create a scatter plot of the first two principal components
scatter_df = pd.DataFrame(transformed_data, columns=[f"PC{i+1}" for i in range(top_components)])
scatter_fig = px.scatter(
    scatter_df,
    x="PC1",
    y="PC2",
    title="Scatter Plot of Top 2 Principal Components",
    labels={"PC1": "Principal Component 1", "PC2": "Principal Component 2"}
)
scatter_fig.show()

Top Contributing Features per Principal Component:
   Principal Component Top Contributing Feature
0                  PC1                HyWi_B(m)
1                  PC2                       nO
2                  PC3                 F02[C-N]
3                  PC4                       Me
4                  PC5                B01[C-Br]
5                  PC6                   NssssC
6                  PC7                B03[C-Cl]
7                  PC8                   nArNO2
8                  PC9                     nCrt
9                 PC10                    N-073
10                PC11                     nN-N
11                PC12                    nHDon
12                PC13                 Psi_i_1d
13                PC14                    N-073
14                PC15                     nN-N
15                PC16                    SdssC
16                PC17                  nArCOOR
17                PC18                    Class


In [39]:
#5) Clustering to find patterns in data
#5a) K-Means clustering (plot for 1-30 clusters, plot elbow plot)

# Range of cluster numbers to evaluate
cluster_range = range(1, 61)
inertia_values = []

# Perform K-means for each cluster number and compute inertia
for k in cluster_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10, max_iter = 800) # can do a max_iter determination later
    kmeans.fit(scaled_data)
    inertia_values.append(kmeans.inertia_)

# Create the elbow plot using Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(cluster_range),
    y=inertia_values,
    mode='lines+markers',
    marker=dict(size=8),
    name='Inertia'
))

fig.update_layout(
    title="Elbow Plot for K-Means Clustering",
    xaxis_title="Number of Clusters",
    yaxis_title="Inertia (Sum of Squared Distances)",
    title_x=0.5,
    template="plotly_white"
)

fig.show()

In [ ]:
#5b) Plot a dendrogram for hierarchial clustering